In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

df = pd.read_csv("data.csv")
df = df.iloc[:, [0, 2, 5, 7, 8, 9, 58, 59, 60, 61, 26]]
df

,Timestamp,What is your current class standing?,What gender do you identify as?,Who do you live with?,"Do you currently live in a house, apartnment, or dorm?",How many people live in your household?,Do you currently work?,How many hours do you work per week on average?,Do you work on or off campus?,Do you work in a department related to your major?,Do you have roommates that are part of your major?
0,2/9/2024 20:12:14,Senior,Male,Neither,House,6,Yes,5 - 10,Off-campus,No,No
1,2/9/2024 20:16:34,Junior,Female,Both,Apartment,4,No,NaN,NaN,NaN,Yes
2,2/9/2024 20:18:55,Junior,Female,Friends,House,4,No,NaN,NaN,NaN,No
3,2/9/2024 20:24:00,Senior,Male,Neither,Apartment,1,No,NaN,NaN,No,No
4,2/9/2024 20:26:16,Graduate,Male,Neither,Apartment,1,Yes,10 - 20,Off-campus,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
255,2/14/2024 19:46:28,Junior,Male,Friends,House,5,Yes,10 - 20,On-campus,No,No
256,2/15/2024 0:28:38,NaN,Male,Family,Apartment,North District 4 bed 2 bath,No,NaN,NaN,NaN,No
257,2/15/2024 8:33:45,Senior,Male,Family,House,9,No,1 - 5,Off-campus,No,No
258,2/15/2024 16:10:40,Sophomore,Female,Family,Apartment,4,Yes,5 - 10,On-campus,No,No


In [2]:
df['How many people live in your household?'] = (df['How many people live in your household?']
                                                 .fillna(0)
                                                 .replace('4 in total', '4')
                                                 .replace('4 (Including me)', '4')
                                                 .replace('at school 4 including me ', '4')
                                                 .replace('3 excluding me', '3')
                                                 .replace('5 including me', '5')
                                                 .replace('North District 4 bed 2 bath', '4')
                                                 .replace('3 (room), 8 (hall), ~70 (building)', '3')
                                                 .astype(int))